In [1]:
###### This program is used to run all the cameras at the sametime at the same framerate ######
# https://docs.luxonis.com/en/latest/pages/faq/

import os
import cv2
import time
import numpy as np
import depthai as dai
from datetime import datetime


# Creates a pipeline for extracting infomation from the Stereo Camera
def createNewPipeline(rgb=True, left=True, right=True, stereo=True):
    
    # Creates pipeline from dai API
    pipeline = dai.Pipeline()
    
    # If an rgb camera is in use then set it up
    if rgb:
        # Create RGB cam pipline
        camRgb = pipeline.create(dai.node.ColorCamera)
        
        # Set the correct socket to communicate with the pipeline and set resolution
        camRgb.setBoardSocket(dai.CameraBoardSocket.RGB)
        camRgb.setResolution(dai.ColorCameraProperties.SensorResolution.THE_1080_P)
        # camRgb.setVideoSize(1920, 1080)
        
        # Link pipeline to output node
        xoutRgb = pipeline.create(dai.node.XLinkOut)
        xoutRgb.setStreamName("rgb")
        camRgb.video.link(xoutRgb.input)

    if left:
        monoLeft = pipeline.create(dai.node.MonoCamera)

        monoLeft.setResolution(dai.MonoCameraProperties.SensorResolution.THE_400_P)
        monoLeft.setBoardSocket(dai.CameraBoardSocket.LEFT)

        xoutLeft = pipeline.create(dai.node.XLinkOut)
        xoutLeft.setStreamName("left")
        monoLeft.out.link(xoutLeft.input)

    if right:
        monoRight = pipeline.create(dai.node.MonoCamera)

        monoRight.setResolution(dai.MonoCameraProperties.SensorResolution.THE_400_P)
        monoRight.setBoardSocket(dai.CameraBoardSocket.RIGHT)

        xoutRight = pipeline.create(dai.node.XLinkOut)
        xoutRight.setStreamName("right")
        monoRight.out.link(xoutRight.input)
        
    if stereo:
        # Create a pipeline for developing the stereo depth
        stereo = pipeline.create(dai.node.StereoDepth)
        
        # Create configuration 
        stereo.initialConfig.setConfidenceThreshold(255)
        stereo.setLeftRightCheck(True)
        stereo.setSubpixel(False)
        
        monoLeft.out.link(stereo.left)
        monoRight.out.link(stereo.right)
        
        xoutDepth = pipeline.create(dai.node.XLinkOut)
        xoutDepth.setStreamName("depth")
        stereo.depth.link(xoutDepth.input)

    device = dai.Device(pipeline)

    output = []

    if rgb:
        rgbVideo = device.getOutputQueue(name="rgb", maxSize=1, blocking=False)
        output.append(rgbVideo)

    if left:
        leftVideo = device.getOutputQueue(name="left", maxSize=1, blocking=False)
        output.append(leftVideo)

    if right:
        rightVideo = device.getOutputQueue(name="right", maxSize=1, blocking=False)
        output.append(rightVideo)
        
    if stereo:
        # dispQ = device.getOutputQueue(name="disp", maxSize=1, blocking=False)
        depthQueue = device.getOutputQueue(name="depth", maxSize=1, blocking=False)
        output.append(depthQueue)
        output.append(stereo)

    output.append(device)
    
    return output

In [2]:
fourcc = cv2.VideoWriter_fourcc('m', 'p', '4', 'v')

RECORD = True

output = createNewPipeline()

rgbCam = output[0]
leftCam = output[1]
rightCam = output[2]
thermalCam = cv2.VideoCapture(0)
stereoCam = output[3]

stereo = output[4]

rgbFrame = rgbCam.get().getCvFrame()
leftFrame = leftCam.get().getCvFrame()
rightFrame = rightCam.get().getCvFrame()
ret, Thermalframe = thermalCam.read()
stereoFrame = stereoCam.get().getCvFrame()

print("rgbFrame:", rgbFrame.shape)
print("leftFrame:", leftFrame.shape)
print("rightFrame:", rightFrame.shape)
print("Thermalframe:", Thermalframe.shape)
print("stereoFrame:", stereoFrame.shape)

FPS = 10
timeNow = str(datetime.now())[:-7].replace(" ", "_").replace(":", "-")
print(timeNow)

if RECORD:
    base = f"videos/{timeNow}"
    os.mkdir(base)
    os.mkdir(f"{base}/processed")

    rgbout = cv2.VideoWriter(f"{base}/rgbout.mp4", fourcc, FPS, (rgbFrame.shape[1], rgbFrame.shape[0]))
    leftout = cv2.VideoWriter(f"{base}/leftout.mp4", fourcc, FPS, (leftFrame.shape[1], leftFrame.shape[0]), 0)
    rightout = cv2.VideoWriter(f"{base}/rightout.mp4", fourcc, FPS, (rightFrame.shape[1], rightFrame.shape[0]), 0)
    thermalout = cv2.VideoWriter(f"{base}/thermalout.mp4", fourcc, FPS, (Thermalframe.shape[1], Thermalframe.shape[0]), 0)
    stereoout = cv2.VideoWriter(f"{base}/stereoout.mp4", fourcc, FPS, (stereoFrame.shape[1], stereoFrame.shape[0]), 0)

print(stereoFrame.shape[1], stereoFrame.shape[0])

#time.sleep(7)
t0 = datetime.now()

while(True):
    
    # Determine the initial frame time 
    initialFrameTime = datetime.now()
          
    rgbFrame = rgbCam.get().getCvFrame()
    leftFrame = leftCam.get().getCvFrame()
    rightFrame = rightCam.get().getCvFrame()
    ret, Thermalframe = thermalCam.read()
    stereoFrame = stereoCam.get().getFrame()
    
    
    Thermalframe = cv2.cvtColor(Thermalframe, cv2.COLOR_BGR2GRAY)
    stereoFrame[0:5,0:5] = 30000 # MAX
    stereoFrame = cv2.normalize(stereoFrame, None, 255, 0, cv2.NORM_INF, cv2.CV_8UC1)
    # stereoFrame = cv2.equalizeHist(stereoFrame)
    # stereoFrame = cv2.applyColorMap(stereoFrame, cv2.COLORMAP_JET)
    
    if RECORD:
        rgbout.write(rgbFrame)
        leftout.write(leftFrame)
        rightout.write(rightFrame)
        thermalout.write(Thermalframe)
        stereoout.write(stereoFrame)
    
    rgbFrame = cv2.resize(rgbFrame, (960, 540))
    cv2.imshow('rgbFrame', rgbFrame)
    cv2.imshow('leftFrame', leftFrame)
    cv2.imshow('rightFrame', rightFrame)
    cv2.imshow('thermalCam', Thermalframe)
    cv2.imshow('stereoFrame', stereoFrame)
    
    lateFrameDifference = float( str(datetime.now() - initialFrameTime)[7:] )
    
    while (lateFrameDifference < (1/FPS)):
        lateFrameDifference = float( str(datetime.now() - initialFrameTime)[7:] )
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

        
t1 = datetime.now()

if RECORD:
    rgbout.release()
    leftout.release()
    rightout.release()
    thermalout.release()
    stereoout.release()
    thermalCam.release()

print(t1 - t0)

cv2.destroyAllWindows()

rgbFrame: (1080, 1920, 3)
leftFrame: (400, 640)
rightFrame: (400, 640)
Thermalframe: (512, 640, 3)
stereoFrame: (400, 640)
2022-12-08_12-49-00
640 400
0:00:17.650750


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

%run utils/superimposition/functions.ipynb
%run utils/extraction/objClass.ipynb

In [ ]:
fourcc = cv2.VideoWriter_fourcc('m', 'p', '4', 'v')

RECORD = True

output = createNewPipeline()

rgbCam = output[0]
leftCam = output[1]
rightCam = output[2]
thermalCam = cv2.VideoCapture(0)
stereoCam = output[3]

stereo = output[4]

rgbFrame = rgbCam.get().getCvFrame()
leftFrame = leftCam.get().getCvFrame()
rightFrame = rightCam.get().getCvFrame()
ret, Thermalframe = thermalCam.read()
stereoFrame = stereoCam.get().getCvFrame()

print("rgbFrame:", rgbFrame.shape)
print("leftFrame:", leftFrame.shape)
print("rightFrame:", rightFrame.shape)
print("Thermalframe:", Thermalframe.shape)
print("stereoFrame:", stereoFrame.shape)

FPS = 10
timeNow = str(datetime.now())[:-7].replace(" ", "_").replace(":", "-")
print(timeNow)

if RECORD:
    base = f"videos/{timeNow}"
    os.mkdir(base)
    os.mkdir(f"{base}/processed")

    rgbout = cv2.VideoWriter(f"{base}/rgbout.mp4", fourcc, FPS, (rgbFrame.shape[1], rgbFrame.shape[0]))
    leftout = cv2.VideoWriter(f"{base}/leftout.mp4", fourcc, FPS, (leftFrame.shape[1], leftFrame.shape[0]), 0)
    rightout = cv2.VideoWriter(f"{base}/rightout.mp4", fourcc, FPS, (rightFrame.shape[1], rightFrame.shape[0]), 0)
    thermalout = cv2.VideoWriter(f"{base}/thermalout.mp4", fourcc, FPS, (Thermalframe.shape[1], Thermalframe.shape[0]), 0)
    stereoout = cv2.VideoWriter(f"{base}/stereoout.mp4", fourcc, FPS, (stereoFrame.shape[1], stereoFrame.shape[0]), 0)

print(stereoFrame.shape[1], stereoFrame.shape[0])

#time.sleep(7)
t0 = datetime.now()

while(True):
    
    # Determine the initial frame time 
    initialFrameTime = datetime.now()
          
    rgbFrame = rgbCam.get().getCvFrame()
    leftFrame = leftCam.get().getCvFrame()
    rightFrame = rightCam.get().getCvFrame()
    ret, Thermalframe = thermalCam.read()
    stereoFrame = stereoCam.get().getFrame()
    
    if calibration_count % 5 == 0:
        scales, offset = calibrate_images(images.copy(), seg_model, config.minimal_img)

        if scales is not None and offset is not None:
            if ten_scales[0] == [1, 1, 1]:
                del ten_scales[0]
                del ten_offsets[0]

            ten_scales.append(scales)
            ten_offsets.append(offset)

        if len(ten_scales) > 30:
            random_delete = int(np.random.rand() * 30)
            
            del ten_scales[random_delete]
            del ten_offsets[random_delete]

        scales, offset = filter_offsets_scales(ten_scales, ten_offsets)
        scales = {"RGB": scales[0], "Thermal": scales[1], "Depth": scales[2]}
        offset = {"RGB": offset[0], "Thermal": offset[1], "Depth": offset[2]}

    # Convert stereo frame to grayscale for visualization
    images["Depth"] = cv2.cvtColor(images["Depth"], cv2.COLOR_BGR2GRAY)

    # Modify images based on calibration settings
    images["Unrelated"] = superimpose_images(images.copy(), add_color=True)
    images = modify_images(images.copy(), scales, offset, images[config.minimal_img].shape[0])

    images["Related"] = superimpose_images(images.copy(), add_color=True)

    stereo_frame_mapped = cv2.normalize(images["Depth"], None, 255, 0, cv2.NORM_INF, cv2.CV_8UC1)
    stereo_frame_mapped = cv2.equalizeHist(stereo_frame_mapped)
    stereo_frame_mapped = cv2.applyColorMap(stereo_frame_mapped, cv2.COLORMAP_JET)
    images["Color_Depth"] = stereo_frame_mapped

    # Create video files for processed output
    if calibration_count == 0:
        thermal_out = cv2.VideoWriter(f"{config.processed_path}Thermal_sii.mp4", config.fourcc, config.fps, (images[config.minimal_img].shape[:2][::-1]))
        stereo_out = cv2.VideoWriter(f"{config.processed_path}Depth_sii.mp4", config.fourcc, config.fps, (images[config.minimal_img].shape[:2][::-1]), 0)
        rgb_out = cv2.VideoWriter(f"{config.processed_path}RGB_sii.mp4", config.fourcc, config.fps, (images[config.minimal_img].shape[:2][::-1]))
        related_sii_out = cv2.VideoWriter(f"{config.processed_path}Related_SII.mp4", config.fourcc, config.fps, (images[config.minimal_img].shape[:2][::-1]))
        unrelated_sii_out = cv2.VideoWriter(f"{config.processed_path}Unrelated_SII.mp4", config.fourcc, config.fps, (images[config.minimal_img].shape[:2][::-1]))

    # Resize related and unrelated images
    images["Related"] = cv2.resize(images["Related"], images[config.minimal_img].shape[:2][::-1])
    images["Unrelated"] = cv2.resize(images["Unrelated"], images[config.minimal_img].shape[:2][::-1])

    # Display images
    for label in images:
        cv2.imshow(label, images[label])

    # Write frames to output video files
    thermal_out.write(images["Thermal"])
    stereo_out.write(images["Depth"])
    rgb_out.write(images["RGB"])
    related_sii_out.write(images["Related"])
    unrelated_sii_out.write(images["Unrelated"])

    # Increment frame counters
    calibration_count += 1
    frame_count += 1

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
print("\nProcessing Ended. Saving videos...")
        
# Release video files
rgb_out.release()
thermal_out.release()
stereo_out.release()
related_sii_out.release()
unrelated_sii_out.release()

print("Saving complete!")

cv2.destroyAllWindows()